# Model Evaluation

## Set Up

Import libraries/packages + preprocessed data + models

In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../"))
from src.data_utils import get_models, get_data
from src.config import BASE_PATH
from src.eval import evaluate_models

import pandas as pd
import warnings

Import notebook globals/data/models

In [ ]:
## Data
DATA_DICT = {"base": get_data(is_nomo=False), "nomo": get_data(is_nomo=True)}

## Models
model_prefix_list = ["DNN", "KNN", "LightGBM", "NLR", "stack", "SVC"]
MODEL_DICT = get_models(model_prefix_list)

# Where to put results
SAVE_PATH = BASE_PATH / "v2_results"

In [ ]:
all_models_test_dict = {}

rows = []
index = []
class_report_dict = evaluate_models(
    model_dict=MODEL_DICT,
    data_dict=DATA_DICT,
    results_path=SAVE_PATH,
    show_cm=False,
    n_bins=4,
    show_roc=False,
    show_cal=True,
    n_bootstraps=5000,
    show_progress=False,
)
## ONLY export test (have access to train/val if need be)
for model, metrics in class_report_dict["test"].items():
    rows.append(metrics)
    index.append(f"ORN_{model}")
all_models_outcomes_df = pd.DataFrame(rows, index=index)

Export test stats

In [ ]:
report_path = SAVE_PATH / "tables" / "class_report" / "class_report.csv"
if report_path.exists():
    warnings.warn(f"Over-writing class report table at path: {report_path}")
    report_path.unlink()
report_path.parent.mkdir(exist_ok=True, parents=True)
all_models_outcomes_df.to_csv(report_path)